In [46]:
# NOTE: change this to the path in your setup
korsmit_exp1_path = "../../data/Korsmit/Exp1/"

In [47]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np

In [48]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/larger_clap_music")
processor = AutoProcessor.from_pretrained("laion/larger_clap_music")

In [49]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [50]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


# Process audio

In [51]:
audio_stimuli = []
stimuli_path = korsmit_exp1_path+"Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [ ]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [ ]:
print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [ ]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)

['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [ ]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# NOTE: currently using only dimensional_captions_induced
tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
tag_embeds = model.get_text_features(**tag_inputs)

# Generate outputs

In [ ]:
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([59, 3])


## Load csv files and extract related columns

In [ ]:
IDim_path = korsmit_exp1_path+"Data/IDim/"
IDim_responses = []

all_dfs = []

for file in os.listdir(IDim_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

IDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59


# Evaluate

In [ ]:
# Find the min and max values in the current sims tensor
old_min = sims.min()
old_max = sims.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

Scaled sims shape: torch.Size([59, 3])
Scaled sims (first 5 rows):
tensor([[7.1646, 7.0362, 6.7361],
        [5.4811, 5.3204, 4.9815],
        [3.6491, 3.4734, 3.0879],
        [3.1602, 3.1239, 2.7137],
        [7.9248, 7.7499, 7.4512],
        [8.4446, 8.2235, 7.9176],
        [2.8711, 2.7444, 2.4005],
        [4.6984, 4.6478, 4.2677],
        [6.2722, 6.0907, 5.6706],
        [5.0021, 4.9391, 4.5388]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000



In [ ]:
human_ratings_tensor = torch.tensor([
    IDim_responses['positive_vector'],
    IDim_responses['relaxed_vector'],
    IDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

Human ratings tensor shape: torch.Size([59, 3])
Human ratings tensor (first 5 rows):
tensor([[4.4872, 4.8798, 4.5185],
        [4.7682, 5.3057, 4.7414],
        [5.3105, 5.7312, 5.0760],
        [5.2582, 5.2651, 5.4488],
        [3.8897, 3.9540, 5.3549],
        [4.3197, 3.8900, 5.5089],
        [4.9825, 5.1875, 5.5354],
        [5.0934, 4.8657, 5.8208],
        [4.7605, 4.8502, 5.4158],
        [5.2577, 4.8688, 5.7003]])



In [ ]:
from scipy.stats import pearsonr # For Pearson correlation

# Comparison Method 1: Mean Absolute Error (MAE)
# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings: {mae:.4f}\n")

# Comparison Method 2: Mean Absolute Percentage Error (MAPE)
# MAPE measures the accuracy of a forecasting method in terms of percentage.
# Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%

# Calculate the absolute percentage error for each element
# Since human responses are between 1 and 9, division by zero is not a concern.
absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100

# Calculate the mean of these percentage errors
mape = torch.mean(absolute_percentage_error)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")

# Comparison Method 3: Root Mean Squared Error (RMSE)
# RMSE measures the square root of the average of the squared differences between predicted and actual values.
# It gives a relatively high weight to large errors.
# Formula: RMSE = sqrt(mean((Actual - Forecast)^2))

# Calculate the squared differences
squared_differences = (human_ratings_tensor - scaled_sims)**2

# Calculate the mean of the squared differences (Mean Squared Error - MSE)
mse = torch.mean(squared_differences)

# Calculate the square root to get RMSE
rmse = torch.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse:.4f}\n")

# Comparison Method 4: Pearson Correlation Coefficient (per column)
# Pearson correlation measures the linear relationship between two sets of data.
# We'll calculate it for each of the three columns (dimensions).

correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings):")
print(f"  Positive Dimension: {correlation_positive:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed:.4f}")
print(f"  Awake Dimension:    {correlation_awake:.4f}")

average_correlation = (correlation_positive + correlation_relaxed + correlation_awake) / 3
print(f"  Average Correlation: {average_correlation:.4f}")

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
print("  valence =", r2_valence)

r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
print("  tension =", r2_tension)

r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
print("  energy =", r2_energy)


Mean Absolute Error (MAE) between scaled_sims and human_ratings: 1.8040

Mean Absolute Percentage Error (MAPE): 39.74%

Root Mean Squared Error (RMSE): 2.1785

Pearson Correlation Coefficients (between scaled_sims and human_ratings):
  Positive Dimension: 0.4015
  Relaxed Dimension:  0.3448
  Awake Dimension:    0.0842
  Average Correlation: 0.2768

R-squared scores:
  valence = -2.706474781036377
  tension = -2.3388662338256836
  energy = -11.282599449157715
